# Pretraining data from Wind and Solar data for Transfer Learning

In [1]:
import os
import pandas as pd
from google.colab import auth
from datetime import datetime
auth.authenticate_user()
!gcloud source repos clone github_aistream-peelout_flow-forecast --project=gmap-997
os.chdir('/content/github_aistream-peelout_flow-forecast')
!git checkout -t origin/covid_fixes
!python setup.py develop
!pip install -r requirements.txt
!mkdir data
from flood_forecast.trainer import train_function
!pip install git+https://github.com/CoronaWhy/task-geo.git
!wandb login 55c7382c4026d5d2ea3309d36986062e40c75387

$ git clone https://github.com/AIStream-Peelout/flow-forecast
Cloning into '/content/github_aistream-peelout_flow-forecast'...
remote: Total 3769 (delta 2433), reused 3769 (delta 2433)
Receiving objects: 100% (3769/3769), 2.68 MiB | 11.34 MiB/s, done.
Resolving deltas: 100% (2433/2433), done.
Project [gmap-997] repository [github_aistream-peelout_flow-forecast] was cloned to [/content/github_aistream-peelout_flow-forecast].
Branch 'covid_fixes' set up to track remote branch 'covid_fixes' from 'origin'.
Switched to a new branch 'covid_fixes'
/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:454: UserWarning: Normalizing '0.01dev' to '0.1.dev0'
  warnings.warn(tmpl.format(**locals()))
running develop
running egg_info
creating flood_forecast.egg-info
writing flood_forecast.egg-info/PKG-INFO
writing dependency_links to flood_forecast.egg-info/dependency_links.txt
writing requirements to flood_forecast.egg-info/requires.txt
writing top-level names to flood_forecast.egg-info/top_leve

In [2]:
# Get wind data
!wget https://storage.googleapis.com/coronaviruspublicdata/forecast_2/wind.csv
!wget https://storage.googleapis.com/coronaviruspublicdata/forecast_2/solar.csv

--2020-05-11 17:04:48--  https://storage.googleapis.com/coronaviruspublicdata/forecast_2/wind.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 2607:f8b0:400c:c0d::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4936771 (4.7M) [text/csv]
Saving to: ‘wind.csv’

wind.csv            100%[===================>]   4.71M  --.-KB/s    in 0.09s   

2020-05-11 17:04:49 (55.2 MB/s) - ‘wind.csv’ saved [4936771/4936771]

--2020-05-11 17:04:52--  https://storage.googleapis.com/coronaviruspublicdata/forecast_2/solar.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 2607:f8b0:400c:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6156494 (5.9M) [text/csv]
Saving to: ‘solar.csv’

solar.csv           100%[===================>] 

## Pre training on Wind Data

In [0]:
import pandas as pd
wind = pd.read_csv('wind.csv')
wind['datetime'] = pd.to_datetime(wind['time']).dt.date
wind.set_index('datetime', drop=False)
wind.head()

,time,AT,BE,BG,CH,CZ,DE,DK,EE,ES,FI,FR,EL,HR,HU,IE,IT,LT,LU,LV,NL,NO,PL,PT,RO,SI,SK,SE,UK,datetime
0,1986-01-01 00:00:00,0.047786,0.023020,0.048940,0.065907,0.041685,0.031583,0.017365,0.014149,0.079043,0.005800,0.049822,0.051933,0.030507,0.025005,0.011889,0.046250,0.035880,0.014839,0.019004,0.014293,0.010351,0.029919,0.076675,0.029107,0.015193,0.054001,0.017463,0.030419,1986-01-01
1,1986-01-02 00:00:00,0.045921,0.036297,0.067995,0.077502,0.026427,0.023506,0.014981,0.015682,0.119019,0.007176,0.063090,0.115133,0.035716,0.039431,0.016575,0.051848,0.016988,0.019510,0.013771,0.020373,0.006469,0.031359,0.106900,0.044379,0.024623,0.034362,0.008086,0.022146,1986-01-02
2,1986-01-03 00:00:00,0.067308,0.021352,0.101287,0.103680,0.057274,0.046181,0.023478,0.009570,0.106574,0.004687,0.048678,0.123855,0.051901,0.029249,0.054734,0.062773,0.017350,0.056217,0.011871,0.010782,0.007217,0.027554,0.160308,0.047235,0.032093,0.023788,0.010004,0.060345,1986-01-03
3,1986-01-04 00:00:00,0.043833,0.050756,0.039337,0.075418,0.025843,0.025011,0.020003,0.008595,0.135060,0.004102,0.092991,0.089767,0.055547,0.044340,0.016779,0.055305,0.019638,0.055925,0.013604,0.030366,0.007998,0.025986,0.208236,0.037510,0.028663,0.018115,0.009546,0.030981,1986-01-04
4,1986-01-05 00:00:00,0.082394,0.014302,0.033055,0.090867,0.065186,0.028168,0.016261,0.009780,0.095232,0.005172,0.045049,0.074312,0.081576,0.082401,0.038972,0.102499,0.020079,0.018873,0.013913,0.012728,0.007241,0.047764,0.115451,0.037254,0.057101,0.072843,0.013872,0.023346,1986-01-05


In [0]:
# Getting the real countries' name from 2 letter code

!pip install pycountry
import pycountry
names = {}
for code in wind.columns:
    try:
        names[code] = pycountry.countries.get(alpha_2=code).name
    except:
        print(code)

# For some reason, these two were not present
names['EL'] = 'Greece'
names['UK'] = 'United Kingdom'

     |████████████████████████████████| 10.0MB 2.7MB/s 
  Created wheel for pycountry: filename=pycountry-19.8.18-py2.py3-none-any.whl size=10627361 sha256=cc9554e8cf6849198cb62e2d68fa6b20b9df33d86087e0184e32b029a6370e70
  Stored in directory: /root/.cache/pip/wheels/a2/98/bf/f0fa1c6bf8cf2cbdb750d583f84be51c2cd8272460b8b36bd3
Successfully built pycountry
time
EL
UK
datetime


In [0]:
wind.rename(columns = names, inplace=True)
wind['year'] = pd.to_datetime(wind['time']).map(lambda x: x.year)
wind['month'] = pd.to_datetime(wind['time']).map(lambda x: x.month)
wind['weekday'] = pd.to_datetime(wind['time']).map(lambda x: x.weekday())

In [0]:
# Making seperate dataframes for each country's data and saving in seperate CSV files
!mkdir wind
country_wise = {}
for country in names.values():
    country_wise[country] = wind[['datetime', 'year', 'month', 'weekday', country]]
    country_wise[country].to_csv('wind/'+country+'.csv')
country_wise[list(country_wise.keys())[5]].head()

,datetime,year,month,weekday,Germany
0,1986-01-01,1986,1,2,0.031583
1,1986-01-02,1986,1,3,0.023506
2,1986-01-03,1986,1,4,0.046181
3,1986-01-04,1986,1,5,0.025011
4,1986-01-05,1986,1,6,0.028168


In [0]:
wind.tail()

,time,Austria,Belgium,Bulgaria,Switzerland,Czechia,Germany,Denmark,Estonia,Spain,Finland,France,Greece,Croatia,Hungary,Ireland,Italy,Lithuania,Luxembourg,Latvia,Netherlands,Norway,Poland,Portugal,Romania,Slovenia,Slovakia,Sweden,United Kingdom,datetime,year,month,weekday
10952,2015-12-27 00:00:00,0.119231,0.077324,0.153964,0.134614,0.081821,0.055697,0.015100,0.038722,0.127358,0.013060,0.102966,0.163328,0.108146,0.044005,0.017154,0.112251,0.014419,0.092283,0.030101,0.019930,0.004189,0.034805,0.170225,0.136308,0.127401,0.047375,0.008486,0.028426,2015-12-27,2015,12,6
10953,2015-12-28 00:00:00,0.128963,0.087688,0.151576,0.125772,0.054648,0.063808,0.024135,0.040206,0.103774,0.011533,0.089861,0.164480,0.096811,0.063488,0.010358,0.121303,0.010499,0.091828,0.036198,0.067833,0.004630,0.023683,0.059809,0.118209,0.129720,0.048307,0.008479,0.020515,2015-12-28,2015,12,0
10954,2015-12-29 00:00:00,0.094872,0.028177,0.070883,0.121686,0.056463,0.054009,0.006732,0.029413,0.126492,0.008377,0.077485,0.170938,0.071934,0.074617,0.011983,0.119000,0.017571,0.031521,0.012752,0.042996,0.004232,0.039898,0.173905,0.033584,0.064835,0.057409,0.011607,0.055220,2015-12-29,2015,12,1
10955,2015-12-30 00:00:00,0.086713,0.054810,0.100528,0.114044,0.097525,0.041072,0.006402,0.014740,0.123499,0.003810,0.091785,0.064106,0.076847,0.132149,0.027625,0.108127,0.027267,0.014551,0.031294,0.057475,0.003595,0.067484,0.065125,0.106734,0.026071,0.134139,0.009297,0.011749,2015-12-30,2015,12,2
10956,2015-12-31 00:00:00,0.111772,0.095547,0.115314,0.031523,0.122601,0.029161,0.012323,0.019210,0.092276,0.004629,0.067331,0.077266,0.108809,0.131938,0.022887,0.081279,0.022490,0.078602,0.018948,0.071013,0.003991,0.091865,0.087822,0.132066,0.114436,0.131965,0.015428,0.045504,2015-12-31,2015,12,3


In [0]:
# Config file for WanDB sweeps

def make_config_file(file_path, df_len):
  run = wandb.init(project="pretrain-wind")
  wandb_config = wandb.config
  train_number = df_len * .7
  validation_number = df_len *.9
  config_default={                 
    "model_name": "MultiAttnHeadSimple",
    "model_type": "PyTorch",
    "model_params": {
      "number_time_series":4,
      "seq_len":wandb_config["forecast_history"], 
      "output_seq_len":wandb_config["out_seq_length"],
      "forecast_length":wandb_config["out_seq_length"]
     },
    "dataset_params":
    {  "class": "default",
       "training_path": file_path,
       "validation_path": file_path,
       "test_path": file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"],
       "forecast_length":wandb_config["out_seq_length"],
       "train_end": int(train_number),
       "valid_start":int(train_number+1),
       "valid_end": int(validation_number),
       "target_col": [file_path.split('.')[0].split('/')[1]],
       "relevant_cols": [file_path.split('.')[0].split('/')[1], "month", "weekday", "year"],
       "scaler": "StandardScaler", 
       "interpolate": False
    },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "Adam",
       "optim_params":
       {

       },
       "lr": wandb_config["lr"],
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    
    },
    "GCS": False,
    
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE"],
   "inference_params":
   {     
         "datetime_start":"2010-01-01",
          "hours_to_forecast":2000, 
          "test_csv_path":file_path,
          "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
          "dataset_params":{
             "file_path": file_path,
             "forecast_history":wandb_config["forecast_history"],
             "forecast_length":wandb_config["out_seq_length"],
             "relevant_cols": [file_path.split('.')[0].split('/')[1], "month", "weekday", "year"],
             "target_col": [file_path.split('.')[0].split('/')[1]],
             "scaling": "StandardScaler",
             "interpolate_param": False
          }
      }
  }
  wandb.config.update(config_default)
  return config_default

sweep_config = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "batch_size": {
            "values": [2]
        },
        "lr":{
            "values":[0.001]
        },
        "forecast_history":{
            "values":[1, 2]
        },
        "out_seq_length":{
            "values":[1, 2]
        }
    }
}

Run sweep

In [0]:
import os
import pandas as pd

# The countries we will be pretraining our wind data on
os.listdir('wind')

['Lithuania.csv',
 'Bulgaria.csv',
 'Ireland.csv',
 'Netherlands.csv',
 'Slovenia.csv',
 'Croatia.csv',
 'Spain.csv',
 'Germany.csv',
 'Romania.csv',
 'Greece.csv',
 'Hungary.csv',
 'Portugal.csv',
 'Belgium.csv',
 'Austria.csv',
 'Norway.csv',
 'Poland.csv',
 'United Kingdom.csv',
 'Latvia.csv',
 'France.csv',
 'Denmark.csv',
 'Switzerland.csv',
 'Finland.csv',
 'Estonia.csv',
 'Italy.csv',
 'Luxembourg.csv',
 'Sweden.csv',
 'Slovakia.csv',
 'Czechia.csv']

In [0]:
import wandb
for country in os.listdir('wind'):
    file_path = 'wind/'+country
    full_len = len(pd.read_csv(file_path))
    sweep_id = wandb.sweep(sweep_config, project="pretrain-wind")
    wandb.agent(sweep_id, lambda:train_function("PyTorch", make_config_file(file_path, full_len)))
    !gsutil cp -n -r model_save gs://coronaviruspublicdata/pretrained

Believe me, the above cell executed. It just took... 13 hours to do so.

**Check out the sweeps here :**  *https://app.wandb.ai/pranjalya/pretrain-wind*

## Pre Training on Solar Data

In [0]:
import pandas as pd
solar = pd.read_csv('solar.csv')

In [27]:
solar.head()

,LocalTime,Power(MW),Power(MW).1,Power(MW).2,Power(MW).3,Power(MW).4,Power(MW).5,Power(MW).6,Power(MW).7,Power(MW).8,Power(MW).9,Power(MW).10,Power(MW).11,Power(MW).12,Power(MW).13,Power(MW).14,Power(MW).15,Power(MW).16,Power(MW).17,Power(MW).18,Power(MW).19,Power(MW).20,Power(MW).21,Power(MW).22,Power(MW).23,Power(MW).24,Power(MW).25,Power(MW).26,Power(MW).27,Power(MW).28,Power(MW).29,Power(MW).30,Power(MW).31,Power(MW).32,Power(MW).33,Power(MW).34,Power(MW).35,Power(MW).36,Power(MW).37,Power(MW).38,...,Power(MW).97,Power(MW).98,Power(MW).99,Power(MW).100,Power(MW).101,Power(MW).102,Power(MW).103,Power(MW).104,Power(MW).105,Power(MW).106,Power(MW).107,Power(MW).108,Power(MW).109,Power(MW).110,Power(MW).111,Power(MW).112,Power(MW).113,Power(MW).114,Power(MW).115,Power(MW).116,Power(MW).117,Power(MW).118,Power(MW).119,Power(MW).120,Power(MW).121,Power(MW).122,Power(MW).123,Power(MW).124,Power(MW).125,Power(MW).126,Power(MW).127,Power(MW).128,Power(MW).129,Power(MW).130,Power(MW).131,Power(MW).132,Power(MW).133,Power(MW).134,Power(MW).135,Power(MW).136
0,2006-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2006-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2006-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2006-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2006-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
solar.describe()

,Power(MW),Power(MW).1,Power(MW).2,Power(MW).3,Power(MW).4,Power(MW).5,Power(MW).6,Power(MW).7,Power(MW).8,Power(MW).9,Power(MW).10,Power(MW).11,Power(MW).12,Power(MW).13,Power(MW).14,Power(MW).15,Power(MW).16,Power(MW).17,Power(MW).18,Power(MW).19,Power(MW).20,Power(MW).21,Power(MW).22,Power(MW).23,Power(MW).24,Power(MW).25,Power(MW).26,Power(MW).27,Power(MW).28,Power(MW).29,Power(MW).30,Power(MW).31,Power(MW).32,Power(MW).33,Power(MW).34,Power(MW).35,Power(MW).36,Power(MW).37,Power(MW).38,Power(MW).39,...,Power(MW).97,Power(MW).98,Power(MW).99,Power(MW).100,Power(MW).101,Power(MW).102,Power(MW).103,Power(MW).104,Power(MW).105,Power(MW).106,Power(MW).107,Power(MW).108,Power(MW).109,Power(MW).110,Power(MW).111,Power(MW).112,Power(MW).113,Power(MW).114,Power(MW).115,Power(MW).116,Power(MW).117,Power(MW).118,Power(MW).119,Power(MW).120,Power(MW).121,Power(MW).122,Power(MW).123,Power(MW).124,Power(MW).125,Power(MW).126,Power(MW).127,Power(MW).128,Power(MW).129,Power(MW).130,Power(MW).131,Power(MW).132,Power(MW).133,Power(MW).134,Power(MW).135,Power(MW).136
count,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,...,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.00000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000
mean,97.990021,72.601543,72.617678,181.308368,163.551646,64.378909,68.242027,118.852452,79.612140,101.382236,75.003927,77.286043,69.262551,78.501012,73.466770,79.956670,78.386968,73.687637,72.940055,73.225892,77.151800,77.681207,77.547582,77.543021,76.565895,76.831962,76.478326,78.114215,77.010734,70.163134,77.808933,70.796005,70.287329,77.684585,76.008676,76.927332,77.182888,72.178669,77.444770,77.269050,...,73.652795,69.244050,72.905161,177.831687,64.941169,73.224571,54.165501,143.012654,78.924091,69.42296,68.952023,54.896039,75.234688,53.857613,64.792473,61.921416,72.592370,54.449314,72.350617,77.925634,72.908951,151.620405,41.334774,77.789300,70.294016,75.423577,41.880676,76.978515,163.736540,76.747531,77.040929,76.639506,123.436848,77.488529,76.994170,123.925892,75.624348,72.612363,61.697565,74.114180
std,121.215824,100.775117,102.617019,255.577980,227.711359,91.088087,96.013131,167.768777,109.091165,142.448961,105.413903,106.748239,96.573719,108.095548,103.759917,109.660217,111.123343,104.098793,103.392740,103.677634,108.577382,108.463883,108.651308,106.827237,105.974013,107.632612,105.833609,110.312326,107.852568,98.031765,109.073624,96.789302,97.835354,108.632086,106.472926,107.922032,106.690069,102.596689,107.988948,107.989447,...,92.000652,97.306594,100.894989,216.932139,91.735845,103.566030,75.384950,198.830916,108.327914,96.93925,96.811196,76.699214,105.744341,74.698423,91.477063,85.882681,100.702548,76.392924,101.918631,109.124838,103.196428,186.609687,57.095260,109.048518,96.184135,106.200268,57.693641,108.015106,228.709291,106.271689,107.825797,105.532143,170.698819,109.003895,107.316424,170.578983,93.098493,103.116330,86.162491,104.574143
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [29]:
solar.tail()

,LocalTime,Power(MW),Power(MW).1,Power(MW).2,Power(MW).3,Power(MW).4,Power(MW).5,Power(MW).6,Power(MW).7,Power(MW).8,Power(MW).9,Power(MW).10,Power(MW).11,Power(MW).12,Power(MW).13,Power(MW).14,Power(MW).15,Power(MW).16,Power(MW).17,Power(MW).18,Power(MW).19,Power(MW).20,Power(MW).21,Power(MW).22,Power(MW).23,Power(MW).24,Power(MW).25,Power(MW).26,Power(MW).27,Power(MW).28,Power(MW).29,Power(MW).30,Power(MW).31,Power(MW).32,Power(MW).33,Power(MW).34,Power(MW).35,Power(MW).36,Power(MW).37,Power(MW).38,...,Power(MW).97,Power(MW).98,Power(MW).99,Power(MW).100,Power(MW).101,Power(MW).102,Power(MW).103,Power(MW).104,Power(MW).105,Power(MW).106,Power(MW).107,Power(MW).108,Power(MW).109,Power(MW).110,Power(MW).111,Power(MW).112,Power(MW).113,Power(MW).114,Power(MW).115,Power(MW).116,Power(MW).117,Power(MW).118,Power(MW).119,Power(MW).120,Power(MW).121,Power(MW).122,Power(MW).123,Power(MW).124,Power(MW).125,Power(MW).126,Power(MW).127,Power(MW).128,Power(MW).129,Power(MW).130,Power(MW).131,Power(MW).132,Power(MW).133,Power(MW).134,Power(MW).135,Power(MW).136
5827,2006-08-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5828,2006-08-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5829,2006-08-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5830,2006-08-31 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5831,2006-08-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
solar.rename(columns={'LocalTime':'datetime'}, inplace=True)
solar = solar[['datetime', 'Power(MW)']]
solar.set_index('datetime', drop=False)

,datetime,Power(MW)
datetime,,
2006-03-03 06:00:00,2006-03-03 06:00:00,10.5
2006-03-03 07:00:00,2006-03-03 07:00:00,202.0
2006-03-03 08:00:00,2006-03-03 08:00:00,295.3
2006-03-03 09:00:00,2006-03-03 09:00:00,331.3
2006-03-03 10:00:00,2006-03-03 10:00:00,337.1
...,...,...
2006-08-31 13:00:00,2006-08-31 13:00:00,111.3
2006-08-31 14:00:00,2006-08-31 14:00:00,201.1
2006-08-31 15:00:00,2006-08-31 15:00:00,203.8


So, basically the first 1469 values, and last 6 values are just 0, so let's remove those rows, as they may just add bias.

In [34]:
solar = solar.iloc[1470:5826]
solar.describe()

,Power(MW)
count,2886.000000
mean,110.812751
std,123.216462
min,0.000000
25%,0.000000
50%,42.900000
75%,232.750000
max,365.900000


In [0]:
# Extracting the timeseries relevant columns
solar['day'] = pd.to_datetime(solar['datetime']).map(lambda x: x.day)
solar['month'] = pd.to_datetime(solar['datetime']).map(lambda x: x.month)
solar['weekday'] = pd.to_datetime(solar['datetime']).map(lambda x: x.weekday())
solar['hour'] = pd.to_datetime(solar['datetime']).map(lambda x: x.hour)

In [39]:
solar.to_csv('selected_solar.csv')
solar.tail()

,datetime,Power(MW),day,month,weekday,hour
5821,2006-08-31 13:00:00,111.3,31,8,3,13
5822,2006-08-31 14:00:00,201.1,31,8,3,14
5823,2006-08-31 15:00:00,203.8,31,8,3,15
5824,2006-08-31 16:00:00,135.5,31,8,3,16
5825,2006-08-31 17:00:00,52.9,31,8,3,17


In [0]:
# Config file for WanDB sweeps

def make_config_file(file_path, df_len):
  run = wandb.init(project="pretrain-wind")
  wandb_config = wandb.config
  train_number = df_len * .7
  validation_number = df_len *.9
  config_default={                 
    "model_name": "MultiAttnHeadSimple",
    "model_type": "PyTorch",
    "model_params": {
      "number_time_series":5,
      "seq_len":wandb_config["forecast_history"], 
      "output_seq_len":wandb_config["out_seq_length"],
      "forecast_length":wandb_config["out_seq_length"]
     },
    "dataset_params":
    {  "class": "default",
       "training_path": file_path,
       "validation_path": file_path,
       "test_path": file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"],
       "forecast_length":wandb_config["out_seq_length"],
       "train_end": int(train_number),
       "valid_start":int(train_number+1),
       "valid_end": int(validation_number),
       "target_col": ["Power(MW)"],
       "relevant_cols": ["Power(MW)", "month", "weekday", "hour", "day"],
       "scaler": "StandardScaler", 
       "interpolate": False
    },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "Adam",
       "optim_params":
       {

       },
       "lr": wandb_config["lr"],
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    
    },
    "GCS": False,
    
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE"],
   "inference_params":
   {     
         "datetime_start":"2006-08-22",
          "hours_to_forecast":150, 
          "test_csv_path":file_path,
          "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
          "dataset_params":{
             "file_path": file_path,
             "forecast_history":wandb_config["forecast_history"],
             "forecast_length":wandb_config["out_seq_length"],
             "relevant_cols": ["Power(MW)", "month", "weekday", "hour", "day"],
             "target_col": ["Power(MW)"],
             "scaling": "StandardScaler",
             "interpolate_param": False
          }
      }
  }
  wandb.config.update(config_default)
  return config_default

sweep_config = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "batch_size": {
            "values": [2, 3, 4]
        },
        "lr":{
            "values":[0.001, 0.01]
        },
        "forecast_history":{
            "values":[1, 2, 3, 5, 8]
        },
        "out_seq_length":{
            "values":[1, 2]
        }
    }
}

In [47]:
# Running the sweep
import wandb
file_path = 'selected_solar.csv'
full_len = len(pd.read_csv(file_path))
sweep_id = wandb.sweep(sweep_config, project="pretrain-solar")
wandb.agent(sweep_id, lambda:train_function("PyTorch", make_config_file(file_path, full_len)))
!gsutil cp -n -r model_save gs://coronaviruspublicdata/pretrained

Create sweep with ID: wyhhj2a6
Sweep URL: https://app.wandb.ai/pranjalya/pretrain-solar/sweeps/wyhhj2a6
wandb: Agent Starting Run: j15aa4jk with config:
	batch_size: 2
	forecast_history: 1
	lr: 0.001
	out_seq_length: 1
wandb: Agent Started Run: j15aa4jk


wandb: WARNING Ignoring project='pretrain-wind' passed to wandb.init when running a sweep


interpolate should be below
Now loading and scaling selected_solar.csv
interpolate should be below
Now loading and scaling selected_solar.csv
interpolate should be below
Now loading and scaling selected_solar.csv
Using Wandb config:
wandb_version: 1

GCS:
  desc: null
  value: false
_wandb:
  desc: null
  value:
    cli_version: 0.8.35
    framework: torch
    is_jupyter_run: true
    is_kaggle_kernel: false
    python_version: 3.6.9
batch_size:
  desc: null
  value: 2
dataset_params:
  desc: null
  value:
    batch_size: 2
    class: default
    forecast_history: 1
    forecast_length: 1
    interpolate: false
    relevant_cols:
    - Power(MW)
    - month
    - weekday
    - hour
    - day
    scaler: StandardScaler
    target_col:
    - Power(MW)
    test_path: selected_solar.csv
    train_end: 2020
    training_path: selected_solar.csv
    valid_end: 2597
    valid_start: 2021
    validation_path: selected_solar.csv
forecast_history:
  desc: null
  value: 1
forward_params:
  desc: 

/content/github_aistream-peelout_flow-forecast/flood_forecast/evaluator.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preds'] = 0
/content/github_aistream-peelout_flow-forecast/flood_forecast/evaluator.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preds'][history_length:] = end_tensor.numpy().tolist()
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:1042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

torch.Size([150])
test_data scale
Un-transforming data


...

torch.Size([150])
test_data scale
Un-transforming data


/content/github_aistream-peelout_flow-forecast/flood_forecast/evaluator.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preds'][history_length:] = end_tensor_list
/content/github_aistream-peelout_flow-forecast/flood_forecast/trainer.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_acc = evaluate_model(trained_model, model_type, params["dataset_params"]["target_col"], params["metrics"], params["inference_params"], {})


Current historical dataframe
      Unnamed: 0            datetime  ...  original_index      preds
2644        5584 2006-08-21 16:00:00  ...            2644   0.000000
2645        5585 2006-08-21 17:00:00  ...            2645   0.000000
2646        5586 2006-08-21 18:00:00  ...            2646   0.000000
2647        5587 2006-08-21 19:00:00  ...            2647   0.000000
2648        5588 2006-08-21 20:00:00  ...            2648   0.000000
...          ...                 ...  ...             ...        ...
2797        5737 2006-08-28 01:00:00  ...            2797 -12.428269
2798        5738 2006-08-28 02:00:00  ...            2798 -11.582688
2799        5739 2006-08-28 03:00:00  ...            2799 -35.455238
2800        5740 2006-08-28 04:00:00  ...            2800   4.994682
2801        5741 2006-08-28 05:00:00  ...            2801   4.506309

[158 rows x 9 columns]


/usr/local/lib/python3.6/dist-packages/plotly/matplotlylib/mpltools.py:368: MatplotlibDeprecationWarning:


The is_frame_like function was deprecated in Matplotlib 3.1 and will be removed in 3.3.

/usr/local/lib/python3.6/dist-packages/plotly/matplotlylib/mplexporter/exporter.py:84: UserWarning:

Blended transforms not yet supported. Zoom behavior may not work as expected.

/usr/local/lib/python3.6/dist-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/usr/local/lib/python3.6/dist-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/usr/local/lib/python3.6/dist-packages/plotly/matplotlylib/mpltools.py:368: MatplotlibDeprecationWarning:


The is_frame_like function was deprecated in Matplotlib 3.1 and will be removed in 3.3.

/usr/local/lib/python3.6/dist-packages/plotly/matplotlylib/renderer

wandb: Agent Finished Run: y0jtd5ru 

Copying file://model_save/11_May_202006_23PM_model.pth [Content-Type=application/octet-stream]...
Copying file://model_save/11_May_202006_12PM_model.pth [Content-Type=application/octet-stream]...
Copying file://model_save/11_May_202006_23PM.json [Content-Type=application/json]...
Copying file://model_save/11_May_202006_26PM_model.pth [Content-Type=application/octet-stream]...
\
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://model_save/11_May_202006_38PM_model.pth [Content-Type=application/octet-stream]...
Copying file://model_save/11_May_202006_18PM.json [Content-Type=application/json]...
Copying file://model_save/11_May_202006_16PM_model.pth [Content-Type=application/octet-stream]...
Copying file://model_save/11_May_202005_57PM.jso

Similarly, this cell ran too.

**Check out the sweeps here :**  *https://app.wandb.ai/pranjalya/pretrain-solar*